In [ ]:
!pip install torch==2.6.0 -q cassio datasets langchain openai tiktoken

In [ ]:
!pip install langchain_community

In [ ]:
!pip install langchain

In [ ]:
from langchain.vectorstores.cassandra import Cassandra

In [ ]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

In [ ]:
%pip install langchain_openai

In [ ]:
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
from datasets import load_dataset
import cassio

In [ ]:
!pip install PyPDF2

In [ ]:
from PyPDF2 import PdfReader

In [ ]:
from google.colab import userdata

In [ ]:
Astra_DB_token = userdata.get('token')
astr_db_id =  userdata.get('id')
OpenAI_API_KEY = userdata.get('OPENAI_API')

In [ ]:
pdfreader = PdfReader("budget_speech.pdf")

In [ ]:
from typing_extensions import Concatenate
raw_text = ""
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

In [ ]:
raw_text

In [ ]:
cassio.init(token = Astra_DB_token,database_id = astr_db_id)

In [ ]:
llm = ChatOpenAI(openai_api_key = OpenAI_API_KEY,base_url = "https://models.inference.ai.azure.com",model="gpt-4o")
embedding  = OpenAIEmbeddings(openai_api_key = OpenAI_API_KEY,base_url = "https://models.inference.ai.azure.com",model = "text-embedding-3-small")

In [ ]:
astra_vector_store = Cassandra(
    embedding = embedding,
    table_name = "qa_mini_demo",
    session = None,
    keyspace = None,
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
texts[:50]

In [ ]:
astra_vector_store.add_texts(texts[:50])

In [ ]:
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [ ]:
input_query = "How much the agriculture target will be increased to and what the focus wil be"
answer = astra_vector_index.query(input_query,llm=llm).strip()
answer